##  Make datasets

### 1. import packages

In [1]:
import numpy as np
from scipy import interpolate
import matplotlib.pyplot as plt
from osgeo import gdal
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
import torchvision
from torch.utils.data import Dataset
import os
from tqdm import tqdm, trange

ModuleNotFoundError: No module named 'scipy'

folder path

In [7]:
path_folder = '/Volumes/HydesT7/Grad Project/Data/AHN4_preprocessed/'
# for item in os.listdir(path_folder):
#   if item.endswith(('.tif', '.TIF')):
#     print(item)

In [3]:
no_zone = [1, 2, 3, 4, 5, 6, 7, 8, 9]
dict_dtm_30 = {}
dict_dsm_30 = {}
dict_dsm_30_fill = {}
dict_30_Aspect = {}
dict_30_Roughness = {}
dict_30_Slope = {}
dict_30_TPI = {}
dict_30_TRI = {}
for i in no_zone:
  dict_dtm_30[i]='DTM_zone'+str(i)+'_30.tif'
  dict_dsm_30[i]='DSM_zone'+str(i)+'_30.tif'
  dict_dsm_30_fill[i]='DSM_zone'+str(i)+'_30_filled.tif'
  dict_30_Aspect[i]='zone'+str(i)+'_30_Aspect.tif'
  dict_30_Roughness[i]='zone'+str(i)+'_30_Roughness.tif'
  dict_30_Slope[i]='zone'+str(i)+'_30_Slope.tif'
  dict_30_TPI[i]='zone'+str(i)+'_30_TPI.tif'
  dict_30_TRI[i]='zone'+str(i)+'_30_TRI.tif'

In [18]:
def readTIFF(path, tiff, flag=False):
  data = gdal.Open(path + tiff, gdal.GA_ReadOnly)
  band = data.GetRasterBand(1)
  arr = band.ReadAsArray()
  if flag:
    for i in range(arr.shape[0]):
      for j in range(arr.shape[1]):
        if arr[i][j]<-9998:
          arr[i][j]=0.0
    return arr
  else:
    return np.ma.masked_where(arr < -9998, arr)

def normalizeRela(arr):
  arr_mean = np.mean(arr)
  arr_std = np.std(arr)
  return (arr-arr_mean)/arr_std

def slicePic(pic, cut_width, cut_length):
  sliced_pic = []
  width = pic.data.shape[0]
  length = pic.data.shape[1]
  num_width = int(width/cut_width)
  num_length = int(length/cut_length)
  for i in range(0, num_width):
    for j in range(0, num_length):
      # dtm_ = dtm[i*cut_width:(i+1)*cut_width, j*cut_length:(j+1)*cut_length]
      # dsm_ = dsm[i*cut_width:(i+1)*cut_width, j*cut_length:(j+1)*cut_length]
      pic_ = pic[i*cut_width:(i+1)*cut_width, j*cut_length:(j+1)*cut_length]
      # mean_pic_ = np.mean(pic_[:, :, 0])
      # std_pic_ = np.std(pic_[:, :, 0])
      # pic_[:, :, 0] = (pic_[:, :, 0]-mean_pic_)/std_pic_
      sliced_pic.append(pic_)
  return sliced_pic

def augmentateData(pics):
  pics_augmentated = []
  for pic in pics:
    pic_rot90 = np.rot90(pic)
    pic_rot180 = np.rot90(pic, 2)
    pic_rot270 = np.rot90(pic, 3)
    pic_flip = np.fliplr(pic)
    pic_flip_rot90 = np.rot90(pic_flip)
    pic_flip_rot180 = np.rot90(pic_flip, 2)
    pic_flip_rot270 = np.rot90(pic_flip, 3)
    pics_augmentated += [pic_rot90, pic_rot180, pic_rot270, pic_flip, pic_flip_rot90, pic_flip_rot180, pic_flip_rot270]
  return pics_augmentated
  

In [5]:
def generateGroundTruth(dsm, dtm):
  length = dsm.shape[0]
  width = dsm.shape[1]
  mask_ = np.zeros((2, length, width), dtype=bool)
  for i in range(length):
    for j in range(width):
      mask_[0][i][j] = True
      if dsm.mask[i][j] == False:
        if dtm.mask[i][j] == True:
          mask_[1][i][j] = True
          mask_[0][i][j] = False
        elif dsm.data[i][j] - dtm.data[i][j] > 2.0:
          mask_[1][i][j] = True
          mask_[0][i][j] = False
  return mask_


In [19]:
def stackPics(dsm_fill, slope, layer, normalization=True):
  pics = []
  for n in range(len(dsm_fill)):
    if normalization:
      pics.append(np.stack((normalizeRela(dsm_fill[n]), normalizeRela(slope[n]), normalizeRela(layer[n])), axis=2))
  arr_pics = np.stack(pics)
  arr_pics = arr_pics.astype(np.float32)
  return arr_pics

### 3. Making dataset with different sizes

#### 1. 64x64

In [7]:
cut_length = 64
cut_width = 64

dsm_64 = []
dtm_64 = []
ground_truth_64 = []
dsm_fill_64 = []
aspect_64 = []
roughness_64 = []
slope_64 = []
tpi_64 = []
tri_64 = []

for n in tqdm(no_zone):
  dsm = readTIFF(path_folder, dict_dsm_30[n])
  dtm = readTIFF(path_folder, dict_dtm_30[n])
  dsm_fill = readTIFF(path_folder, dict_dsm_30_fill[n], flag=True)
  aspect = readTIFF(path_folder, dict_30_Aspect[n], flag=True)
  roughness = readTIFF(path_folder, dict_30_Roughness[n], flag=True)
  slope = readTIFF(path_folder, dict_30_Slope[n], flag=True)
  tpi = readTIFF(path_folder, dict_30_TPI[n], flag=True)
  tri = readTIFF(path_folder, dict_30_TRI[n], flag=True)
  dsm_64 = dsm_64+slicePic(dsm, cut_length, cut_width)
  dtm_64 = dtm_64+slicePic(dtm, cut_length, cut_width)
  dsm_fill_64 = dsm_fill_64+slicePic(dsm_fill, cut_length, cut_width)
  aspect_64 = aspect_64+slicePic(aspect, cut_length, cut_width)
  roughness_64 = roughness_64+slicePic(roughness, cut_length, cut_width)
  slope_64 = slope_64+slicePic(slope, cut_length, cut_width)
  tpi_64 = tpi_64+slicePic(tpi, cut_length, cut_width)
  tri_64 = tri_64+slicePic(tri, cut_length, cut_width)

dsm_64 = dsm_64 + augmentateData(dsm_64)
dtm_64 = dtm_64 + augmentateData(dtm_64)
dsm_fill_64 = dsm_fill_64 + augmentateData(dsm_fill_64)
aspect_64 = aspect_64 + augmentateData(aspect_64)
roughness_64 = roughness_64 + augmentateData(roughness_64)
slope_64 = slope_64 + augmentateData(slope_64)
tpi_64 = tpi_64 + augmentateData(tpi_64)
tri_64 = tri_64 + augmentateData(tri_64)


for nn in trange(len(dsm_64)):
  ground_truth_64.append(generateGroundTruth(dsm_64[nn], dtm_64[nn]))


100%|██████████| 27672/27672 [08:46<00:00, 52.55it/s]


In [20]:
Pics_aspect = stackPics(dsm_fill_64, slope_64, aspect_64)
Pics_roughness = stackPics(dsm_fill_64, slope_64, roughness_64)
Pics_tpi = stackPics(dsm_fill_64, slope_64, tpi_64)
Pics_tri = stackPics(dsm_fill_64, slope_64, tri_64)

In [21]:
Pics_aspect.shape

(27672, 64, 64, 3)

In [22]:
Masks = np.stack(ground_truth_64)
Masks = Masks.astype(np.float32)
Masks.shape

(27672, 2, 64, 64)

In [17]:
out_file_path = '/Volumes/T7/Grad Project/Data/U_Net/64_64/'
np.save(out_file_path+'Pics_aspect', Pics_aspect)
np.save(out_file_path+'Pics_roughness', Pics_roughness)
np.save(out_file_path+'Pics_tpi', Pics_tpi)
np.save(out_file_path+'Pics_tri', Pics_tri)
np.save(out_file_path+'Masks', Masks)

### Old codes:

In [37]:
dict_ground_truth = {}
for i in no_zone:
  dsm = readTIFF(path_folder, dict_dsm_30[i])
  dtm = readTIFF(path_folder, dict_dtm_30[i])
  dict_ground_truth[i] = generateGroundTruth(dsm, dtm)

In [30]:
dict_ground_truth[2].shape

(2, 1041, 833)

In [38]:
for i in no_zone:
  count = 0
  for ii in range(dict_ground_truth[i].shape[1]):
    for jj in range(dict_ground_truth[i].shape[2]):
      if dict_ground_truth[i][0][ii][jj] == False:
        count = count + 1
  print(i, dict_ground_truth[i].shape[1]*dict_ground_truth[i].shape[2], count)

1 1943514 169014
2 867153 89300
3 1214514 257834
4 867153 183245
5 1666000 363061
6 3332000 626852
7 2291250 467983
8 971278 352408
9 1457500 400283


In [21]:
mask_ = np.zeros((2, 3, 3), dtype=bool)
mask_

array([[[False, False, False],
        [False, False, False],
        [False, False, False]],

       [[False, False, False],
        [False, False, False],
        [False, False, False]]])

In [19]:
dtm_zone1 = readTIFF(path_folder, dict_dtm_30[1])
# dtm_zone1 = normalizationRela(dtm_zone1)


In [35]:
dtm_zone1

masked_array(
  data=[[0.4448238015174866, 0.47189897298812866, 0.4945898652076721,
         ..., 1.153877854347229, 0.9447280764579773, 1.1161539554595947],
        [0.45900291204452515, 0.4787600040435791, 0.5047914385795593,
         ..., 0.8834675550460815, 0.9006596207618713, 1.0871127843856812],
        [0.47728660702705383, 0.491116464138031, 0.511136531829834, ...,
         1.153686285018921, 1.0460479259490967, 0.8672204613685608],
        ...,
        [1.1860183477401733, 1.2729129791259766, 1.1702306270599365, ...,
         3.2245266437530518, 2.9320554733276367, 2.7083067893981934],
        [1.083808422088623, 0.9614956974983215, 0.7971500754356384, ...,
         3.036719799041748, 2.8023149967193604, 2.6607699394226074],
        [0.7239670753479004, 0.6590371131896973, 0.7602627277374268, ...,
         3.202040910720825, 2.8163774013519287, 2.619251251220703]],
  mask=[[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],

In [20]:
dtm_zone1.shape

(1458, 1333)

In [18]:
display(np.max(dtm_zone1), np.min(dtm_zone1))

29.618162

-11.715275

In [2]:
file_path = '/Volumes/SamDick/Grad Project/Data/ANH4/'
# ndv = 3.4028230607371e+38
# R: DSM, M: DTM
data_r = gdal.Open(file_path + 'R30_16.TIF', gdal.GA_ReadOnly)
# print(data_r.RasterCount)  1
band_r = data_r.GetRasterBand(1)
# band_r.SetNoDataValue(ndv)
# no data value -> 3.4028235e+38
ele_r = band_r.ReadAsArray()

data_rf = gdal.Open(file_path + 'R30_16_filled.TIF', gdal.GA_ReadOnly)
# print(data_r.RasterCount)  1
band_rf = data_rf.GetRasterBand(1)
ele_rf = band_rf.ReadAsArray()

data_m = gdal.Open(file_path + 'M30_16.TIF', gdal.GA_ReadOnly)
# print(data_r.RasterCount)  1
band_m = data_m.GetRasterBand(1)
# band_r.SetNoDataValue(ndv)
# no data value -> 3.4028235e+38
ele_m = band_m.ReadAsArray()


In [3]:
data_tpi = gdal.Open(file_path + 'TPI_30_16.TIF', gdal.GA_ReadOnly)
band_tpi = data_tpi.GetRasterBand(1)
tpi = band_tpi.ReadAsArray()

data_s = gdal.Open(file_path + 'Slope_30_16.TIF', gdal.GA_ReadOnly)
band_s = data_s.GetRasterBand(1)
slope = band_s.ReadAsArray()


In [4]:
for i in range(tpi.shape[0]):
  for j in range(tpi.shape[1]):
    if tpi[i][j]<-9998:
      tpi[i][j]=0.0

for i in range(slope.shape[0]):
  for j in range(slope.shape[1]):
    if slope[i][j]<-9998:
      slope[i][j]=0.0

In [29]:
# count = 0
# for i in range(slope.shape[0]):
#   for j in range(slope.shape[1]):
#     if slope[i][j]<-9998:
#       count += 1
# print((count, slope.shape[0]*slope.shape[1]))

(2998, 556278)


In [5]:
# output[channel] = (input[channel] - mean[channel]) / std[channel]
tpi_mean = np.mean(tpi)
tpi_std = np.std(tpi)
slope_mean = np.mean(slope)
slope_std = np.std(slope)
display((tpi_mean, tpi_std, slope_mean, slope_std))

(-0.00019498802, 1.6129733, 1.8055739, 2.2324755)

In [6]:
tpi = (tpi - tpi_mean)/tpi_std
slope = (slope-slope_mean)/slope_std

In [7]:
pseudo_pic = np.stack((ele_rf, tpi, slope), axis=2)


In [8]:
pic_rot90 = np.rot90(pseudo_pic)
pic_rot180 = np.rot90(pseudo_pic, 2)
pic_rot270 = np.rot90(pseudo_pic, 3)
pic_flip = np.fliplr(pseudo_pic)
pic_flip_rot90 = np.rot90(pic_flip)
pic_flip_rot180 = np.rot90(pic_flip, 2)
pic_flip_rot270 = np.rot90(pic_flip, 3)


In [9]:
masked_dsm = np.ma.masked_where(ele_r < -9998, ele_r)
masked_dtm = np.ma.masked_where(ele_m < -9998, ele_m)


In [10]:
DSM_rot90 = np.rot90(masked_dsm)
DSM_rot180 = np.rot90(masked_dsm, 2)
DSM_rot270 = np.rot90(masked_dsm, 3)
DSM_flip = np.fliplr(masked_dsm)
DSM_flip_rot90 = np.rot90(DSM_flip)
DSM_flip_rot180 = np.rot90(DSM_flip, 2)
DSM_flip_rot270 = np.rot90(DSM_flip, 3)


In [11]:
DTM_rot90 = np.rot90(masked_dtm)
DTM_rot180 = np.rot90(masked_dtm, 2)
DTM_rot270 = np.rot90(masked_dtm, 3)
DTM_flip = np.fliplr(masked_dtm)
DTM_flip_rot90 = np.rot90(DTM_flip)
DTM_flip_rot180 = np.rot90(DTM_flip, 2)
DTM_flip_rot270 = np.rot90(DTM_flip, 3)


In [12]:
pics = [pseudo_pic, pic_rot90, pic_rot180, pic_rot270,
        pic_flip, pic_flip_rot90, pic_flip_rot180, pic_flip_rot270]
dsms = [masked_dsm, DSM_rot90, DSM_rot180, DSM_rot270,
        DSM_flip, DSM_flip_rot90, DSM_flip_rot180, DSM_flip_rot270]
dtms = [masked_dtm, DTM_rot90, DTM_rot180, DTM_rot270,
        DTM_flip, DTM_flip_rot90, DTM_flip_rot180, DTM_flip_rot270]


In [13]:
display(pics[1].shape, dsms[1].shape)


(667, 834, 3)

(667, 834)

In [14]:
def slicePic(dsm, dtm, pic, cut_width, cut_length):
  sliced_dsm = []
  sliced_dtm = []
  sliced_pic = []
  (width, length) = dsm.data.shape
  num_width = int(width/cut_width)
  num_length = int(length/cut_length)
  for i in range(0, num_width):
    for j in range(0, num_length):
      dtm_ = dtm[i*cut_width:(i+1)*cut_width, j*cut_length:(j+1)*cut_length]
      dsm_ = dsm[i*cut_width:(i+1)*cut_width, j*cut_length:(j+1)*cut_length]
      pic_ = pic[i*cut_width:(i+1)*cut_width, j*cut_length:(j+1)*cut_length]
      # mean_pic_ = np.mean(pic_[:, :, 0])
      # std_pic_ = np.std(pic_[:, :, 0])
      # pic_[:, :, 0] = (pic_[:, :, 0]-mean_pic_)/std_pic_
      sliced_dsm.append(dsm_)
      sliced_dtm.append(dtm_)
      sliced_pic.append(pic_)
  return sliced_dsm, sliced_dtm, sliced_pic
  

In [21]:
sliced_dsm = []
sliced_dtm = []
sliced_pic = []
for i in range(len(pics)):
  sliced_dsm_, sliced_dtm_, sliced_pic_ = slicePic(dsms[i], dtms[i], pics[i], 96, 96)
  sliced_dsm = sliced_dsm + sliced_dsm_
  sliced_dtm = sliced_dtm + sliced_dtm_
  sliced_pic = sliced_pic + sliced_pic_

In [22]:
cut_length = 128
cut_width = 128

masks = []
for i in range(len(sliced_dsm)):
  mask_ = np.zeros((2, cut_length, cut_width), dtype=bool)
  for ii in range(sliced_dsm[i].shape[0]):
    for jj in range(sliced_dsm[i].shape[1]):
      if sliced_dsm[i].mask[ii][jj] == True:
        mask_[0][ii][jj] = True
      else:
        if sliced_dtm[i].mask[ii][jj] == True:
            mask_[1][ii][jj] = True
        elif sliced_pic[i][ii][jj][0] - sliced_dtm[i][ii][jj] > 2.0:
            mask_[1][ii][jj] = True
        else:
            mask_[0][ii][jj] = True
  masks.append(mask_)

In [23]:
for i in range(len(sliced_pic)):
  mean_pic_ = np.mean(sliced_pic[i][:, :, 0])
  std_pic_ = np.std(sliced_pic[i][:, :, 0])
  sliced_pic[i][:, :, 0] = (sliced_pic[i][:, :, 0]-mean_pic_)/std_pic_

In [105]:
sliced_pic[0][:, :, 0]

array([[ 1.5745915 ,  1.5704007 ,  1.5592155 , ..., -0.48052517,
        -0.76738364, -1.0623963 ],
       [ 1.5734875 ,  1.5691835 ,  1.5579805 , ..., -0.481203  ,
        -0.7679325 , -1.0628119 ],
       [ 1.6095208 ,  1.6047456 ,  1.5923154 , ...,  0.05394662,
        -0.24605338, -0.55459845],
       ...,
       [-0.19216834, -0.19700867, -0.2098565 , ..., -2.4824982 ,
        -2.7635517 , -3.0602136 ],
       [-0.33638602, -0.3412825 , -0.35427323, ..., -2.7495024 ,
        -3.033569  , -3.3338618 ],
       [-0.48666322, -0.4916265 , -0.50477636, ..., -3.0313585 ,
        -3.3184462 , -3.6223404 ]], dtype=float32)

In [24]:
Pics = np.stack(sliced_pic)
Pics = Pics.astype(np.float32)
Pics.shape

(384, 96, 96, 3)

In [25]:
Masks = np.stack(masks)
Masks = Masks.astype(np.float32)
Masks.shape

(384, 2, 96, 96)

In [26]:
out_file_path = '/Volumes/SamDick/Grad Project/Data/U_Net/96_96/'
np.save(out_file_path+'Pics', Pics)
np.save(out_file_path+'Masks', Masks)

In [19]:
pic = np.zeros((cut_width, cut_length))
sliced_DSM = []
sliced_DSM_f = []
sliced_DTM = []
# count = 0
for i in range(0, num_width):
    for j in range(0, num_length):
        dsm_f = DSM_f[i*cut_width:(i+1)*cut_width,
                      j*cut_length:(j+1)*cut_length]
        dsm = DSM[i*cut_width:(i+1)*cut_width, j*cut_length:(j+1)*cut_length]
#         if not dsm.mask.any():
#             count +=1
# print(count)
        dtm = DTM[i*cut_width:(i+1)*cut_width, j*cut_length:(j+1)*cut_length]
#             count = 0
        layer_ = np.zeros(dsm.shape, dtype=np.float32)
#             dsm_ = np.stack((dsm_, layer_), axis=2)
#             layer_ = np.zeros(dsm_.shape, dtype=float)
#             dsm_ = np.stack((dsm_, layer_), axis=2)
#             dsm_ = np.expand_dims(dsm_, axis=2)
        dsm_f = np.stack((dsm_f, layer_, layer_), axis=2)

#             dtm_ = np.expand_dims(dsm_, axis=2)

        sliced_DSM.append(dsm)
        sliced_DTM.append(dtm)
        sliced_DSM_f.append(dsm_f)


In [45]:
sliced_DSM[0].mask[0][0]


True

In [47]:
masks = []
for i in range(0, len(sliced_DSM)):
    mask_ = np.zeros((2, cut_width, cut_width), dtype=bool)
    for ii in range(sliced_DSM[i].shape[0]):
        for jj in range(sliced_DSM[i].shape[1]):
            #             print(i, ii, jj)
            if sliced_DSM[i].mask[ii][jj] == True:
                mask_[0][ii][jj] = True
            else:
                if sliced_DTM[i][ii][jj] < -9998:
                    mask_[1][ii][jj] = True
                elif sliced_DSM_f[i][ii][jj][0] - sliced_DTM[i][ii][jj] > 2.0:
                    mask_[1][ii][jj] = True
                else:
                    mask_[0][ii][jj] = True
    masks.append(mask_)


In [48]:
masks


[array([[[ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True],
         ...,
         [ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True]],
 
        [[False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False],
         ...,
         [False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False]]]),
 array([[[ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True],
         ...,
         [ True,  True,  True, ...,  True,  True,  True],
         [ True,  True, 

In [34]:
sample_path = file_path + 'sample/'
if not os.path.exists(sample_path):
    os.makedirs(sample_path)

# 6*8=48
cut_width = 48
cut_length = 48
DSM = masked_r
DTM = masked_m
(width, length) = masked_r.data.shape
num_width = int(width/cut_width)
num_length = int(length/cut_length)
display((width, length), (num_width, num_length), num_width*num_length)


(1250, 1000)

(26, 20)

520

In [60]:
pic = np.zeros((cut_width, cut_length))
resp_DSM = []
resp_DTM = []
resp_width = 6
resp_length = 6
num_wid_re = int(cut_width/resp_width)
num_len_re = int(cut_length/resp_length)
# count = 0
for i in range(0, num_width):
    for j in range(0, num_length):
        dsm = DSM[i*cut_width:(i+1)*cut_width, j*cut_length:(j+1)*cut_length]
        if not dsm.mask.any():
            #             count +=1
            # print(count)
            dtm = DTM[i*cut_width:(i+1)*cut_width, j *
                      cut_length:(j+1)*cut_length]
            dsm_ = np.zeros((num_wid_re, num_len_re), dtype=float)
            dtm_ = np.zeros((num_wid_re, num_len_re), dtype=float)
#             count = 0
            for ii in range(0, num_wid_re):
                for jj in range(0, num_len_re):
                    dsm_patch = dsm[ii*resp_width:(ii+1)*resp_width,
                                    jj*resp_length:(jj+1)*resp_length]
                    dsm_[ii][jj] = dsm_patch.mean()
                    dtm_patch = dtm[ii*resp_width:(ii+1)*resp_width,
                                    jj*resp_length:(jj+1)*resp_length]
                    count_ = 0
                    for iii in range(0, resp_width):
                        for jjj in range(0, resp_length):
                            if dtm_patch.mask[iii][jjj] == True:
                                count_ += 1
                    if count_ >= 6:
                        dtm_[ii][jj] = -9999
#                         count += 1
                    else:
                        dtm_[ii][jj] = dtm_patch.mean()
            layer_ = np.zeros(dsm_.shape, dtype=np.float32)
#             dsm_ = np.stack((dsm_, layer_), axis=2)
#             layer_ = np.zeros(dsm_.shape, dtype=float)
#             dsm_ = np.stack((dsm_, layer_), axis=2)
#             dsm_ = np.expand_dims(dsm_, axis=2)
            dsm_ = np.stack((dsm_, layer_, layer_), axis=2)

#             dtm_ = np.expand_dims(dsm_, axis=2)

            resp_DSM.append(dsm_)
            resp_DTM.append(dtm_)
#             print(count)

#             mask_ = np.zeros((cut_width, cut_length), dtype=bool)
#             for ii in range(dsm_.data.shape[0]):
#                 for jj in range(dsm_.data.shape[1]):
#                     if dtm_.mask[ii][jj] == False:
#                         mask_[ii][jj] == True
#                     elif dsm_.data[ii][jj] - dtm_.data[ii][jj] > 2.0:


In [61]:
DSMs = np.stack(resp_DSM)


In [67]:
DSMs = DSMs.astype(np.float32)


In [68]:
DSMs.shape


(195, 8, 8, 3)

In [69]:
masks = []
for i in range(0, len(resp_DSM)):
    mask_ = np.zeros((2, num_wid_re, num_wid_re), dtype=bool)
    for ii in range(resp_DSM[i].shape[0]):
        for jj in range(resp_DSM[i].shape[1]):
            if resp_DTM[i][ii][jj] < -9998:
                mask_[1][ii][jj] = True
            elif resp_DSM[i][ii][jj][0] - resp_DTM[i][ii][jj] > 2.0:
                mask_[1][ii][jj] = True
            else:
                mask_[0][ii][jj] = True
    masks.append(mask_)


In [70]:
Masks = np.stack(masks)


In [73]:
Masks = Masks.astype(np.float32)


In [74]:
Masks.shape


(195, 2, 8, 8)

In [75]:
out_file_path = '/Volumes/SamDick/Grad Project/Data/U_Net/'
np.save(out_file_path+'DSMs', DSMs)
np.save(out_file_path+'Masks', Masks)


In [142]:
masks[133]


array([[[False, False, False, False,  True, False, False, False,  True],
        [ True, False, False, False, False, False, False, False, False],
        [False, False, False, False, False, False, False, False, False],
        [False, False, False, False, False, False, False, False, False],
        [False, False, False, False, False, False, False, False, False],
        [False, False, False, False, False, False, False, False, False],
        [False, False, False, False,  True, False, False, False, False],
        [False, False, False, False, False, False, False, False, False],
        [False, False, False, False, False, False, False, False, False]],

       [[ True,  True,  True,  True, False,  True,  True,  True, False],
        [False,  True,  True,  True,  True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True,  True

In [126]:
resp_DSM[133]


array([[5.88211356, 5.70066791, 3.1650859 , 5.5277621 , 2.40479766,
        5.7149086 , 4.69442156, 5.63392936, 1.66553794],
       [2.07185343, 5.70841895, 3.18577745, 5.1440862 , 6.09836875,
        4.60784446, 5.83231227, 3.99173779, 3.5154722 ],
       [4.25535668, 4.11067581, 6.57765664, 4.06018151, 6.38957426,
        5.01890352, 5.83037186, 5.46660317, 7.05884298],
       [6.73357391, 4.73863559, 6.09194565, 5.51837243, 5.21262953,
        5.9577819 , 5.33015018, 7.01587762, 5.85234112],
       [5.58179601, 6.37199317, 4.8537195 , 6.22620053, 3.72766368,
        5.54059558, 5.35482322, 4.4022895 , 5.06766849],
       [4.03711997, 5.38460244, 4.71515656, 6.21329965, 5.88803143,
        6.8638081 , 6.26359389, 3.38509729, 6.88004218],
       [5.23683463, 7.3972914 , 2.89535692, 5.56428697, 2.45196173,
        3.00737593, 3.83838399, 6.83112505, 5.84194522],
       [8.16592407, 6.08083344, 7.74186622, 6.18744829, 5.51682239,
        7.15699175, 4.96371714, 3.98114522, 3.65990448],


In [125]:
resp_DTM[133]


array([[-9.99900000e+03, -9.99900000e+03,  6.12866879e-01,
         5.18849254e-01,  7.89063656e-01, -9.99900000e+03,
        -9.99900000e+03, -9.99900000e+03,  8.30459595e-01],
       [ 7.47467875e-01,  4.92492795e-01,  7.59106227e-01,
        -9.99900000e+03, -9.99900000e+03, -9.99900000e+03,
        -9.99900000e+03, -9.99900000e+03, -9.99900000e+03],
       [ 7.12436156e-01, -9.99900000e+03, -9.99900000e+03,
        -9.99900000e+03, -9.99900000e+03, -9.99900000e+03,
        -9.99900000e+03, -9.99900000e+03, -9.99900000e+03],
       [-9.99900000e+03, -9.99900000e+03, -9.99900000e+03,
        -9.99900000e+03, -9.99900000e+03, -9.99900000e+03,
        -9.99900000e+03, -9.99900000e+03, -9.99900000e+03],
       [-9.99900000e+03, -9.99900000e+03, -9.99900000e+03,
        -9.99900000e+03, -9.99900000e+03, -9.99900000e+03,
        -9.99900000e+03,  9.20983420e-01, -9.99900000e+03],
       [-9.99900000e+03, -9.99900000e+03, -9.99900000e+03,
        -9.99900000e+03, -9.99900000e+03, -9.999000